In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
# check the current path
print(os.getcwd()) # /content

# You should copy the path : 왼쪽 폴더에서 오른쪽 버튼 후 경로복사
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW')

print(os.getcwd()) # path has been changed

/content
/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW


## 1.라이브러리 설치

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install retinaface-pytorch

In [ ]:
!pip install efficientnet-pytorch

In [ ]:
!pip install albumentations

In [ ]:
!pip install timm

## 2. 데이터셋 준비

metafile.json을 Pandas DataFrame으로 읽기

In [ ]:
import json
import pandas as pd

json_file = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/train_sample_videos/metadata.json'
df = pd.read_json(json_file)
df = df.T

df.head()

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [ ]:
df.loc['aagfhgtpmv.mp4'].label

'FAKE'

## 2-1. 데이터셋 메인 준비

In [ ]:
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from retinaface.pre_trained_models import get_model as get_retinaface_model
import albumentations as A
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import json

# RetinaFace 모델 로드
retinaface_model = get_retinaface_model("resnet50_2020-07-20", max_size=224)
retinaface_model.eval()

# 사용자 데이터셋 경로
train_dir = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/train_sample_videos'
test_dir = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/test_videos'

# JSON 파일을 DataFrame으로 불러와 라벨 정보를 설정
json_file = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/train_sample_videos/metadata.json'
df = pd.read_json(json_file).T  # Transpose to set filenames as index


class RetinaFacePreprocess:
    def __call__(self, image):
        if image is None or image.size == 0:  # image가 비어 있거나 None인 경우 체크
            print("Warning: No face detected and input image is empty. Returning blank image.")
            return np.zeros((244, 244, 3), dtype=np.uint8)  # 빈 이미지 반환 (244x244 크기, RGB 채널)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        faces = retinaface_model.predict_jsons(image)  # 얼굴 검출
        if faces:
            try:
                x1, y1, x2, y2 = faces[0]["bbox"]
                image = image[int(y1):int(y2), int(x1):int(x2)]
            except ValueError:
                pass

        if image.size == 0:  # 얼굴이 검출되지 않아 image가 비어 있는 경우
            print("Warning: No face detected. Using a blank image.")
            return np.zeros((244, 244, 3), dtype=np.uint8)  # 빈 이미지 반환 (244x244 크기, RGB 채널)

        return cv2.resize(image, (244, 244))  # (244, 244) 크기로 리사이즈


# Albumentations 전용 변환 설정
albumentations_transform_train = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.CoarseDropout(num_holes_range=(1, 1),
                    hole_height_range=(10, 20),
                    hole_width_range=(10, 20),
                    fill_value=0,
                    p=0.8),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])
albumentations_transform_val = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

class VideoFrameDataset(Dataset):
    def __init__(self, video_dir, df, preprocess, albumentations_transform=None, num_frames=10, display=False, use_dummy_label=False):
        self.video_dir = video_dir
        self.df = df  # DataFrame을 통해 라벨 설정
        self.preprocess = preprocess
        self.albumentations_transform = albumentations_transform
        self.to_tensor = ToTensor()
        self.num_frames = num_frames
        self.video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
        self.display = display
        self.use_dummy_label = use_dummy_label
        self.frames_and_labels = self._load_frames_and_labels()  # 1500개의 (프레임, 라벨) 리스트 생성

    def _load_frames_and_labels(self):
        frames_and_labels = []
        for video_file in self.video_files:
            video_path = os.path.join(self.video_dir, video_file)
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_interval = max(total_frames // self.num_frames, 1)

            for i in range(self.num_frames):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
                success, frame = cap.read()
                if not success:
                    print(f"Warning: Could not read frame {i} in {video_path}. Skipping.")
                    continue

                # 얼굴 추출 및 전처리
                face = self.preprocess(frame)
                if self.albumentations_transform:
                    face = self.albumentations_transform(image=face)['image']
                face_tensor = self.to_tensor(face)

                # 라벨 설정
                if not self.use_dummy_label:
                    label_name = self.df.loc[video_file, 'label']
                    label = 1 if label_name == 'FAKE' else 0
                else:
                    label = 0  # 더미 라벨 설정

                frames_and_labels.append((face_tensor, label))

                if len(frames_and_labels) >= 1500:  # 총 1500개 항목만 필요
                    break
            cap.release()
            if len(frames_and_labels) >= 1500:
                break
        return frames_and_labels

    def __len__(self):
        return len(self.frames_and_labels)  # 총 1500개의 샘플을 반환

    def __getitem__(self, idx):
        return self.frames_and_labels[idx]  # [3, 244, 244] 크기의 텐서와 float 라벨 반환

# train_dataset과 test_dataset 정의
train_dataset = VideoFrameDataset(
    train_dir, df, preprocess=RetinaFacePreprocess(), albumentations_transform=albumentations_transform_train, display=False
)
test_dataset = VideoFrameDataset(
    test_dir, df, preprocess=RetinaFacePreprocess(), albumentations_transform=albumentations_transform_val, display=True, use_dummy_label=True
)

# train_dataset 개별 데이터 확인
for frames, labels in train_dataset:
    print(f"Single Frame shape: {frames.shape}")  # 예상: [3, 244, 244]
    print(f"Single Label shape: {labels}")  # 예상: 단일 float 값
    break

# DataLoader로 배치 단위 데이터 확인
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 배치 단위 데이터 확인
for frames, labels in train_loader:
    print(f"Batch Frames shape: {frames.shape}")  # 예상: [64, 3, 244, 244]
    print(f"Batch Labels shape: {labels.shape}")  # 예상: [64]
    break

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/hub.py:870: FutureWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile 

Single Frame shape: torch.Size([3, 244, 244])
Single Label shape: 1
Batch Frames shape: torch.Size([64, 3, 244, 244])
Batch Labels shape: torch.Size([64])


1. Ratinaface 결과를 몇개의 이미지에 대해서 imshow로 확인할 것
2. alnumentation 결과도 확인해 imshow로 볼 것
3. label이 (datasize, 2)인지 확인할

In [ ]:
#import os
#os.cpu_count()

In [ ]:
#T4 GPU성능 확인 위해서 적은 거임
#!nvidia-smi

## 3. 데이터로더 준비

In [ ]:
'''
import torch
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

# 데이터셋 인스턴스 생성
dataset = VideoFrameDataset(
    train_dir,
    preprocess=RetinaFacePreprocess(),
    albumentations_transform=albumentations_transform_train
)

# DataLoader 설정 (CPU 연결 시)
dataset_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1)

# 배치 단위로 데이터 확인
for data, labels in dataset_loader:
    print("Data shape:", data.shape)  # (batch_size, num_frames, 3, 224, 224)
    print("Labels shape:", labels.shape)  # (batch_size, 2)

    # 첫 번째 비디오의 첫 번째 프레임을 시각화
    img = data[0][0].permute(1, 2, 0).cpu().numpy()  # 텐서를 이미지로 변환
    plt.imshow(img)
    plt.title("First Frame of First Video in Batch")
    plt.show()
    break

# dataset = VideoFrameDataset(train_dir, preprocess=RetinaFacePreprocess())
# ## 아래는 CPU 연결할 때 사용 : colab에서 CPU 지원 수가 2개이므로, num_workers수도 1~2개
# dataset_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1)
# ## 아래는 GPU 연결할 때 사용 num_workers가 1 또는 2여야 성능이 무리가 없음
# #dataset_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1)
'''

'\nimport torch\nfrom torch.utils.data import DataLoader\nfrom matplotlib import pyplot as plt\n\n# 데이터셋 인스턴스 생성\ndataset = VideoFrameDataset(\n    train_dir,\n    preprocess=RetinaFacePreprocess(),\n    albumentations_transform=albumentations_transform_train\n)\n\n# DataLoader 설정 (CPU 연결 시)\ndataset_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1)\n\n# 배치 단위로 데이터 확인\nfor data, labels in dataset_loader:\n    print("Data shape:", data.shape)  # (batch_size, num_frames, 3, 224, 224)\n    print("Labels shape:", labels.shape)  # (batch_size, 2)\n\n    # 첫 번째 비디오의 첫 번째 프레임을 시각화\n    img = data[0][0].permute(1, 2, 0).cpu().numpy()  # 텐서를 이미지로 변환\n    plt.imshow(img)\n    plt.title("First Frame of First Video in Batch")\n    plt.show()\n    break\n\n# dataset = VideoFrameDataset(train_dir, preprocess=RetinaFacePreprocess())\n# ## 아래는 CPU 연결할 때 사용 : colab에서 CPU 지원 수가 2개이므로, num_workers수도 1~2개\n# dataset_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_worke

In [ ]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_pre_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

## 4. 모델 로드 및 수정

In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary
from efficientnet_pytorch import EfficientNet
import timm  # timm 라이브러리로 Xception 모델 로드

# EfficientNet-B0 모델
class EfficientNetB0(nn.Module):
    def __init__(self, num_classes=1):
        super(EfficientNetB0, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

# Xception 모델
class Xception(nn.Module):
    def __init__(self, num_classes=1):
        super(Xception, self).__init__()
        # timm을 사용하여 Xception 모델 로드
        self.model = timm.create_model('xception', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

# 앙상블 모델
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, x):
        # 두 모델의 예측값을 평균으로 앙상블
        output1 = self.model1(x)
        output2 = self.model2(x)
        return (output1 + output2) / 2  # 평균 예측값

# 모델 인스턴스 생성
effnet_model = EfficientNetB0(num_classes=1)
xception_model = Xception(num_classes=1)

# 앙상블 모델 생성
ensemble_model = EnsembleModel(effnet_model, xception_model)

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

# 모델 요약 출력
summary(ensemble_model, input_size=(3, 224, 224), device='cuda' if torch.cuda.is_available() else 'cpu')

# 학습용 DataLoader 예시
from torch.utils.data import DataLoader

batch_size = 32  # 배치 크기 설정 (GPU 메모리와 학습 속도에 따라 조정)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# 배치 데이터 확인
for frames, labels in train_loader:
    print(f"배치 프레임의 크기: {frames.shape}")  # 예상: [32, 3, 224, 224]
    print(f"배치 레이블의 크기: {labels.shape}")  # 예상: [32]
    break


Loaded pretrained weights for efficientnet-b0


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

## 5. 손실 함수 및 옵티마이저 설정

In [ ]:
import torch.nn as nn
import torch.optim as optim

# 모델 정의 (예: 앙상블 모델 사용)
model = ensemble_model  # 이미 정의된 ensemble_model을 사용

# BCEWithLogitsLoss는 이진 분류에 적합하며, 내부적으로 sigmoid 활성화 함수를 포함하고 있음
criterion = nn.BCEWithLogitsLoss()

# 옵티마이저 설정: Adam + L2 정규화
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)  # L2 정규화 추가 (weight_decay=1e-4)

# 손실 함수와 옵티마이저 확인
print("Loss Function:", criterion)
print("Optimizer:", optimizer)


Loss Function: BCEWithLogitsLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 1e-05
)


## 6. 학습 및 검증 루프 정의

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import random

# Random Erasing 데이터 증강 정의
random_erasing = transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0)

# Mixup 함수 정의
def mixup_data(x, y, alpha=1.0):
    """Mixup 데이터 증강 함수"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)  # Beta 분포에서 샘플링하여 비율 lam 계산
    else:
        lam = 1.0
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)  # 배치 순서 섞기
    mixed_x = lam * x + (1 - lam) * x[index, :]  # 입력 이미지의 가중합
    mixed_y = lam * y + (1 - lam) * y[index]  # 라벨의 가중합
    return mixed_x, mixed_y, lam

# 훈련 루프에서 데이터 증강을 적용하는 함수
def train_one_epoch(model, dataloader, criterion, optimizer, device, alpha=1.0, log_interval=100):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0  # 누적 손실 초기화
    correct = 0  # 맞은 예측 수 초기화
    total = 0  # 총 예측 수 초기화

    # 데이터로더에서 배치별로 데이터를 가져옴
    for i, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)  # 입력 데이터를 장치(GPU 또는 CPU)로 이동
        labels = labels.to(device).unsqueeze(1).float()  # 라벨을 (batch_size, 2) 형식에 맞춰 변환

        # 첫 번째 배치의 몇 개의 샘플을 시각화하여 증강 확인
        if i == 0:  # 첫 번째 배치에만 샘플을 출력
            print("Random Erasing이 적용된 샘플들을 확인합니다:")
            for k in range(min(5, inputs.size(0))):  # 최대 5개의 샘플을 출력
                plt.imshow(inputs[k].cpu().permute(1, 2, 0).numpy())  # 채널 순서를 변경하여 이미지 출력
                plt.show()  # 이미지를 화면에 표시

        # Mixup을 적용하여 새로운 입력과 라벨 생성
        mixed_inputs, mixed_labels, lam = mixup_data(inputs, labels, alpha)

        # 입력 이미지에 랜덤 지우기 증강 적용
        for j in range(mixed_inputs.size(0)):
            mixed_inputs[j] = random_erasing(mixed_inputs[j])  # 각 이미지를 랜덤 지우기 증강 적용

        optimizer.zero_grad()  # 옵티마이저의 경사도 초기화
        outputs = model(mixed_inputs)  # 모델을 통해 예측 수행
        loss = criterion(outputs, mixed_labels)  # 손실 계산
        loss.backward()  # 역전파를 통해 경사도 계산
        optimizer.step()  # 옵티마이저로 가중치 업데이트

        # 예측 결과를 이진 분류에 맞게 0과 1로 변환
        outputs[outputs >= 0.5] = 1
        outputs[outputs < 0.5] = 0

        # 누적 손실 계산
        running_loss += loss.item() * inputs.size(0)
        # 맞춘 예측 수를 합산
        correct += outputs.eq(mixed_labels).int().sum()

    # 평균 손실과 정확도를 반환
    return running_loss / len(dataloader), 100 * correct / len(dataloader.dataset)

# 평가 함수는 Mixup이 적용되지 않으므로 기존 코드와 동일하게 유지
def evaluate(model, dataloader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정
    running_loss = 0.0  # 누적 손실 초기화
    correct = 0  # 맞은 예측 수 초기화
    total = 0  # 총 예측 수 초기화

    with torch.no_grad():  # 평가에서는 경사도를 계산하지 않음
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            labels = labels.unsqueeze(1).float()
            loss = criterion(outputs, labels)

            # 예측 결과를 이진 분류에 맞게 0과 1로 변환
            outputs[outputs >= 0.5] = 1
            outputs[outputs < 0.5] = 0

            # 누적 손실 계산
            running_loss += loss.item() * inputs.size(0)
            # 맞춘 예측 수를 합산
            correct += outputs.eq(labels).int().sum()

    # 평균 손실과 정확도를 반환
    return running_loss / len(dataloader), 100 * correct / len(dataloader.dataset)


## 7. 모델 학습

In [ ]:
import copy
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)  # 이미 정의된 앙상블 모델

# 손실 함수 및 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()  # 이진 분류에 적합한 BCEWithLogitsLoss 사용
optimizer = optim.Adam(ensemble_model.parameters(), lr=1e-4)

# Train과 Test DataLoader 설정
batch_size = 32  # 배치 크기 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)  # num_workers는 데이터 로딩 속도를 최적화
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Train 함수 정의
def train_one_epoch(model, dataloader, criterion, optimizer, device, log_interval=100):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()  # 이진 분류이므로 labels의 형태를 (batch_size, 1)로 맞춤

        optimizer.zero_grad()

        # 모델에 입력 전달
        outputs = model(inputs)
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        running_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # 로그 간격마다 출력
        if (batch_idx + 1) % log_interval == 0:
            print(f'Batch [{batch_idx + 1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy

# Test 함수 정의
def evaluate(model, dataloader, device):
    model.eval()  # 모델을 평가 모드로 설정
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            # `inputs`만 추출하여 `.to(device)` 사용
            inputs = batch[0].to(device)

            outputs = model(inputs)
            preds = (outputs >= 0).float().cpu().numpy()
            predictions.extend(preds)

    return predictions

# 학습 설정
num_epochs = 20
best_val_acc = 0.0
best_model_wts = copy.deepcopy(ensemble_model.state_dict())  # 초기 모델 가중치 저장

for epoch in range(num_epochs):
    # Train 데이터셋 학습 (라벨이 있음)
    train_loss, train_acc = train_one_epoch(ensemble_model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    # Test 데이터셋 예측 (라벨이 없음)
    predictions = evaluate(ensemble_model, test_loader, device)

    # 예측값 출력 (처음 10개만 예시로 표시)
    print(f"Test Predictions (first 10): {predictions[:10]}")

    # 모델의 성능 개선 여부 확인
    if train_acc > best_val_acc:
        best_val_acc = train_acc
        best_model_wts = copy.deepcopy(ensemble_model.state_dict())  # 가장 성능 좋은 가중치 저장

# 가장 성능 좋은 가중치로 모델 업데이트
ensemble_model.load_state_dict(best_model_wts)


Epoch 1/20
Train Loss: 0.0590, Train Acc: 98.0000
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32)]
Epoch 2/20
Train Loss: 0.0313, Train Acc: 98.9333
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32)]
Epoch 3/20
Train Loss: 0.0267, Train Acc: 99.1333
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), arra

<All keys matched successfully>